In [1]:
import pandas as pd
import numpy as np
import csv
from os import system


1\. **Text files**
Perform the following operations on plain txt files:

create a list of integrer numbers and then save it to a text file named data_int.txt. Run the cat command to print the content of the file.
create a matrix of 5x5 floats and then save it to a text file named data_float.txt. Use the cat command to print the content of the file.
load the txt file of the previous point and convert it to a csv file by hand.

In [26]:
#a)
from random import random, seed

L = [ int(random()*10) for i in range(10) ]
print("list l:\n ", L, "\n")

with open("./data_int.txt", mode = 'w') as f:
    for item in L:
        f.write("%s " % item)
        
print("loded txt list file:")  
system("cat data_int.txt")      
!type data_int.txt #because my os is windows instead of cat I had to use type
#!cat data_int.txt
print('\n')
      
#b) 
b= np.random.random((5,5))
mat = np.matrix(b)
print("random 5*5 matrix:\n", mat, "\n")    
with open('data_float.txt','wb') as f:
    #f.write(str(mat))
    for line in mat:
        np.savetxt(f, line, fmt='%.2f')
        
print("data_float:")
!type data_float.txt
#!cat ./data_float.txt 
print('\n')

#c)
f = open('./data_float.txt', 'r')
textContent = f.read()
print("loded txt file: \n", textContent)
f.close()

with open('./data_float.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(textContent)



import zlib
lines = textContent.splitlines()
rawContent = [line.split(' ') for line in lines]
rows = zip(*[rawContent]*1)
with open('data_float.csv', 'w') as out_file:
    writer = csv.writer(out_file)
    for row in rows:
        writer.writerows(row)
print("csv file: ")
!type data_float.csv
#!cat ./data_float.csv #read and show csv file
print('\n')

list l:
  [3, 1, 3, 2, 3, 9, 4, 6, 8, 3] 

loded txt list file:
3 1 3 2 3 9 4 6 8 3 

random 5*5 matrix:
 [[0.66141052 0.98284092 0.95851142 0.27956604 0.54672302]
 [0.41064493 0.1064934  0.52930036 0.16217343 0.20317383]
 [0.29183404 0.80261567 0.81029934 0.92862366 0.73148444]
 [0.35077808 0.13596273 0.46325513 0.66488599 0.69146959]
 [0.30909531 0.79311611 0.93853049 0.40278235 0.83254931]] 


data_float:



data_int.txt


The system cannot find the file specified.
Error occurred while processing: #because.
The system cannot find the file specified.
Error occurred while processing: my.
The system cannot find the file specified.
Error occurred while processing: os.
The system cannot find the file specified.
Error occurred while processing: is.
The system cannot find the file specified.
Error occurred while processing: windows.
The system cannot find the file specified.
Error occurred while processing: instead.
The system cannot find the file specified.
Error occurred while processing: of.
The system cannot find the file specified.
Error occurred while processing: cat.
The system cannot find the file specified.
Error occurred while processing: I.
The system cannot find the file specified.
Error occurred while processing: had.
The system cannot find the file specified.
Error occurred while processing: to.
The system cannot find the file specified.
Error occurred while processing: use.
The sy

0.66 0.98 0.96 0.28 0.55
0.41 0.11 0.53 0.16 0.20
0.29 0.80 0.81 0.93 0.73
0.35 0.14 0.46 0.66 0.69
0.31 0.79 0.94 0.40 0.83


loded txt file: 
 0.66 0.98 0.96 0.28 0.55
0.41 0.11 0.53 0.16 0.20
0.29 0.80 0.81 0.93 0.73
0.35 0.14 0.46 0.66 0.69
0.31 0.79 0.94 0.40 0.83

csv file: 
0.66,0.98,0.96,0.28,0.55

0.41,0.11,0.53,0.16,0.20

0.29,0.80,0.81,0.93,0.73

0.35,0.14,0.46,0.66,0.69

0.31,0.79,0.94,0.40,0.83





2\. **JSON files**
Load the file user_data.json, which can be found at:

https://www.dropbox.com/s/sz5klcdpckc39hd/user_data.json
and filter the data by the "CreditCardType" when it equals to "American Express". Than save the data to a new CSV file.

In [32]:
import json


input_json = json.load(open('user_data.json'))

americanEx_json = [x for x in input_json if x['CreditCardType'] == 'American Express']

df = pd.DataFrame(americanEx_json, columns = ['ID' , 'JobTitle', 'EmailAddress' , 'FirstNameLastName','CreditCard','CreditCardType'])

df.to_csv (r'json_data_americanEx.csv', index = False)

!type json_data_americanEx.csv 


ID,JobTitle,EmailAddress,FirstNameLastName,CreditCard,CreditCardType
2,Investment  Advisor,Clint_Thorpe5003@bulaffy.com,Clint Thorpe,7083-8766-0251-2345,American Express
12,Retail Trainee,Phillip_Carpenter9505@famism.biz,Phillip Carpenter,3657-0088-0820-5247,American Express
28,Project Manager,Russel_Graves1378@extex.org,Russel Graves,6718-4818-8011-6024,American Express
39,Stockbroker,Leanne_Newton1268@typill.biz,Leanne Newton,5438-0816-4166-4847,American Express
57,Budget Analyst,Tony_Giles1960@iatim.tech,Tony Giles,8130-3425-7573-7745,American Express
62,CNC Operator,Owen_Allcott5125@bauros.biz,Owen Allcott,4156-0107-7210-2630,American Express
68,Project Manager,Liam_Lynn3280@kideod.biz,Liam Lynn,7152-3247-6053-2233,American Express
74,Dentist,Regina_Woodcock5820@yahoo.com,Regina Woodcock,0208-1753-3870-8002,American Express
81,HR Specialist,Carter_Wallace9614@atink.com,Carter Wallace,4256-7201-6717-4322,American Express
92,Staffing Consultant,Maia_Stark2797@jiman.org,Maia Stark,385

3. CSV files with Pandas

Load the file from this url:

https://www.dropbox.com/s/kgshemfgk22iy79/mushrooms_categorized.csv
with Pandas.

explore and print the DataFrame
calculate, using groupby(), the average value of each feature, separately for each class
save the file in a JSON format.

In [34]:

data  = pd.read_csv('mushrooms_categorized.csv')

display(data)

class_mean = data.groupby(["class"]).mean()

class_mean.to_json("mushrooms_categorized.json", orient="index")
display(class_mean) 
!type mushrooms_categorized.json 

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,1,5,2,4,1,6,1,0,1,4,...,2,7,7,0,2,1,4,2,3,5
1,0,5,2,9,1,0,1,0,0,4,...,2,7,7,0,2,1,4,3,2,1
2,0,0,2,8,1,3,1,0,0,5,...,2,7,7,0,2,1,4,3,2,3
3,1,5,3,8,1,6,1,0,1,5,...,2,7,7,0,2,1,4,2,3,5
4,0,5,2,3,0,5,1,1,0,4,...,2,7,7,0,2,1,0,3,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8119,0,3,2,4,0,5,0,0,0,11,...,2,5,5,0,1,1,4,0,1,2
8120,0,5,2,4,0,5,0,0,0,11,...,2,5,5,0,0,1,4,0,4,2
8121,0,2,2,4,0,5,0,0,0,5,...,2,5,5,0,1,1,4,0,1,2
8122,1,3,3,4,0,8,1,0,1,0,...,1,7,7,0,2,1,0,7,4,2


,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
class,,,,,,,,,,,,,,,,,,,,,
0,3.266160,1.615970,4.581749,0.653992,4.334601,0.954373,0.285171,0.068441,6.622624,0.615970,...,1.798479,6.098859,6.064639,0.0,1.931559,1.125475,3.007605,3.201521,3.283270,1.148289
1,3.436159,2.055158,4.421859,0.159346,3.940756,0.995403,0.028601,0.567926,2.863636,0.514811,...,1.394280,5.512768,5.504597,0.0,2.002043,1.009193,1.522983,4.021450,4.031665,1.895812


{"0":{"cap-shape":3.2661596958,"cap-surface":1.6159695817,"cap-color":4.5817490494,"bruises":0.6539923954,"odor":4.3346007605,"gill-attachment":0.9543726236,"gill-spacing":0.2851711027,"gill-size":0.0684410646,"gill-color":6.6226235741,"stalk-shape":0.6159695817,"stalk-root":1.4980988593,"stalk-surface-above-ring":1.7756653992,"stalk-surface-below-ring":1.7984790875,"stalk-color-above-ring":6.0988593156,"stalk-color-below-ring":6.0646387833,"veil-type":0.0,"veil-color":1.9315589354,"ring-number":1.1254752852,"ring-type":3.0076045627,"spore-print-color":3.2015209125,"population":3.283269962,"habitat":1.1482889734},"1":{"cap-shape":3.4361593463,"cap-surface":2.0551583248,"cap-color":4.4218590398,"bruises":0.1593462717,"odor":3.9407558733,"gill-attachment":0.9954034729,"gill-spacing":0.0286006129,"gill-size":0.5679264556,"gill-color":2.8636363636,"stalk-shape":0.5148110317,"stalk-root":0.6925434116,"stalk-surface-above-ring":1.3595505618,"stalk-surface-below-ring":1.3942798774,"stalk-colo

4\. **Reading the credit card numbers**

In [35]:

import math as m

with open('credit_card.dat', 'rb') as file:
    file_content = file.readlines()
    allcarts =[]
    word_counter = 0
    word_size = 8 
    total_digitnumber = 16+3 
    digit_length = 6          
    
    
    for row in file_content:
        if(round(len(row)/digit_length) >= total_digitnumber):
            
            utf_string = row.decode("utf-8")  
            utf_string = utf_string[:-4]      

            
            start_index = 0
            card_number=[]
            
            for i in range(total_digitnumber):
                end_index = start_index + digit_length
                digits_4 = utf_string[start_index:end_index]
                card_number.append(digits_4)
                start_index += digit_length    


            credit_card = ""
            for i in range(total_digitnumber):
                credit_card += chr(int(card_number[i],2))
            
            allcarts.append(credit_card)
            
    print("Number of all the Credit Card is :\n") 
    for i in allcarts:
        print(i, "\n")
        
        


Number of all the Credit Card is :

7648 5673 3775 2271 

3257 8247 3354 2266 

2722 0001 4011 6652 

0661 3063 3742 3150 

0432 1608 1462 4742 

5827 2027 8785 7303 

5774 8528 2087 1117 

8140 1210 6352 2845 

5764 1133 7301 7100 

6456 1737 4126 6726 

1228 8631 7382 0000 

7051 0160 5374 3166 

0618 3587 1630 6376 

1545 5454 7444 5636 

6735 3116 3202 6834 

7287 5011 1547 8413 

7033 2607 3328 4200 

2568 5244 1874 5024 

1684 2253 7570 7118 

0672 2576 0575 6631 

6332 8353 8787 1340 

1813 3361 1175 4211 

2477 6450 8840 2368 

5512 3505 2563 1326 

3083 7882 0621 0025 

4521 5148 8045 0334 

7563 3654 8713 5787 

8324 2664 0476 5561 

0565 2504 7168 3510 

5107 5507 1767 0738 

2462 1821 2448 1443 

2788 0638 6861 6554 

5851 5873 5474 0547 

0670 1004 4013 2655 

5874 5506 3048 0806 

2805 5401 8462 1260 

5083 8406 6310 1862 

1076 1445 3013 2266 

8440 4804 4844 5277 

4758 6141 0686 1387 

7586 0675 0315 2568 

2544 1258 7432 5165 

3474 5023 4434 5626 

1410 0270 0434 508

5\. **Write data to a binary file**

a) Start from the `data/data_000637.txt` file that we have used during the previous lectures, and convert it to a binary file according to the format defined below:

*Hints*:
- Read the first 10 lines using Pandas
- Iterate over the DataFrame rows
- For every row, ``pack'' the values (features) into a single 64-bit word, according to the format specified above. Use bit-wise shifts and operators to do so.
- Write each 64-bit word to a binary file. You can use `struct` in this way:
```
binary_file.write( struct.pack('<q', word) )
```
where `word` is the 64-bit word.
- Close the file after completing the loop.

b) Check that the binary file is correctly written by reading it with the code used in the lecture `06_OSEMN.ipynb`, and verify that the content of the `txt` and binary files is consistent.

c) What is the difference of the size on disk between equivalent `txt` and binary files?

In [ ]:
import numpy as np
import pandas as pd
import struct

#a
N = 10
data = pd.read_csv('data_000637.txt', header=0, nrows= N, skiprows = 0)
#print(data)
word_size = 8
print(data)
with open('data/new_binary_data.dat', 'wb') as file:
    for row in data.iterrows():
        word  = 0
        word = word | row[1]['HEAD']
        word = (word << 4) | row[1]['FPGA']
        word = (word << 9) | row[1]['TDC_CHANNEL']
        word = (word << 32) | row[1]['ORBIT_CNT']
        word = (word << 12) | row[1]['BX_COUNTER']
        word = (word << 5) | row[1]['TDC_MEAS']
        packed = struct.pack('<q', word) # get an 8-byte word
        file.write(packed)
file.close()

#b
data = {}
columns = ['HEAD', 'FPGA', 'CHANNEL', 'ORBIT_CNT', 'BX_CNT', 'TDC_MEAS']
df = pd.DataFrame({}, columns=columns)

with open('data/new_binary_data.dat', 'rb') as file:
    file_content = file.read()
    word_counter = 0
    word_size = 8 # size of the word in bytes
    for i in range(0, len(file_content), word_size):
        word_counter += 1
        if word_counter > 10: break
        word = struct.unpack('q', file_content[i : i + word_size])[0] # get an 8-byte word
        head     = (word >> 62) & 0x3
        fpga     = (word >> 58) & 0xF
        tdc_chan = (word >> 49) & 0x1FF
        orb_cnt  = (word >> 17) & 0xFFFFFFFF
        bx       = (word >> 5 ) & 0xFFF
        tdc_meas = (word >> 0 ) & 0x1F
        entry = {'HEAD' : head, 'FPGA' : fpga, 'CHANNEL' : tdc_chan, 'ORBIT_CNT' : orb_cnt, 'BX_CNT' : bx, 'TDC_MEAS' : tdc_meas}
        df = df.append(entry, ignore_index=True)
print(df)



#c
#./data/data_000637.dat: 80 byte
#./data/data_000637.txt: 359 byte
# by saving data in binary format we use less storage.

In [54]:
import numpy as np
import pandas as pd
import struct

#a
N = 10
data = pd.read_csv('5/data/data_000637.txt', header=0, nrows= N, skiprows = 0)
word_size = 8
print(data)
with open('5/data/new_binary_data.dat', 'wb') as file:
    for row in data.iterrows():
        word  = 0
        word = word | row[1]['HEAD']
        word = (word << 4) | row[1]['FPGA']
        word = (word << 9) | row[1]['TDC_CHANNEL']
        word = (word << 32) | row[1]['ORBIT_CNT']
        word = (word << 12) | row[1]['BX_COUNTER']
        word = (word << 5) | row[1]['TDC_MEAS']
        packed = struct.pack('<q', word) # get an 8-byte word
        file.write(packed)
file.close()

#b
data = {}
columns = ['HEAD', 'FPGA', 'CHANNEL', 'ORBIT_CNT', 'BX_CNT', 'TDC_MEAS']
df = pd.DataFrame({}, columns=columns)

with open('5/data/new_binary_data.dat', 'rb') as file:
    file_content = file.read()
    word_counter = 0
    word_size = 8 # size of the word in bytes
    for i in range(0, len(file_content), word_size):
        word_counter += 1
        if word_counter > 10: break
        word = struct.unpack('q', file_content[i : i + word_size])[0] # get an 8-byte word
        head     = (word >> 62) & 0x3
        fpga     = (word >> 58) & 0xF
        tdc_chan = (word >> 49) & 0x1FF
        orb_cnt  = (word >> 17) & 0xFFFFFFFF
        bx       = (word >> 5 ) & 0xFFF
        tdc_meas = (word >> 0 ) & 0x1F
        entry = {'HEAD' : head, 'FPGA' : fpga, 'CHANNEL' : tdc_chan, 'ORBIT_CNT' : orb_cnt, 'BX_CNT' : bx, 'TDC_MEAS' : tdc_meas}
        df = df.append(entry, ignore_index=True)
print(df)



#c
#./data/data_000637.dat: 80 byte
#./data/data_000637.txt: 359 byte
# by saving data in binary format we use less storage.

   HEAD  FPGA  TDC_CHANNEL   ORBIT_CNT  BX_COUNTER  TDC_MEAS
0     1     0          123  3869200167        2374        26
1     1     0          124  3869200167        2374        27
2     1     0           63  3869200167        2553        28
3     1     0           64  3869200167        2558        19
4     1     0           64  3869200167        2760        25
5     1     0           63  3869200167        2762         4
6     1     0           61  3869200167        2772        14
7     1     0          139  3869200167        2776         0
8     1     0           62  3869200167        2774        21
9     1     0           60  3869200167        2788         7
  HEAD FPGA CHANNEL   ORBIT_CNT BX_CNT TDC_MEAS
0    1    0     123  3869200167   2374       26
1    1    0     124  3869200167   2374       27
2    1    0      63  3869200167   2553       28
3    1    0      64  3869200167   2558       19
4    1    0      64  3869200167   2760       25
5    1    0      63  3869200167   2762   

C:\Users\Shayan\AppData\Local\Temp\ipykernel_20124\3685370879.py:43: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(entry, ignore_index=True)
C:\Users\Shayan\AppData\Local\Temp\ipykernel_20124\3685370879.py:43: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(entry, ignore_index=True)
C:\Users\Shayan\AppData\Local\Temp\ipykernel_20124\3685370879.py:43: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(entry, ignore_index=True)
C:\Users\Shayan\AppData\Local\Temp\ipykernel_20124\3685370879.py:43: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(entry, ignore_index=True)
C:\Users\Shayan\AppD